In [47]:
#Imports
import requests as rq
import json
import requests
import urllib2
from bs4 import BeautifulSoup
import re
import string
from afinn import Afinn
import os
import matplotlib.pyplot as plt


In [48]:
#rapgenius information
rap_genius_id = "_17NJOxR9_RGp1XTQ1Vl3O--bxYh-cSzcCwASKlr-Xlyj3vh2_BsdedBZLX6xgdt"
client_secret = "2wlydcjFiqOlSI5Zi65OeizvaWRx16xkkjIBfPkqZ9t1aFNNYTrjza8ny6xYlXdJQSP_vMzj6e20O7ORtjIIcQ"
client_access_token = "Uzv_tj6uOXE9_1eX2wCrOGi3j4JtmYcm1kVF-UmPcC5gJh1MQZKJLph4aFp9frFx"

In [49]:
#pass the URL of rapgenius and get returned the song lyrics
def getLyrics (URL):
    page = requests.get(URL)
    html = BeautifulSoup(page.text, "html.parser") # Extract the page's HTML as a string

    # Scrape and return the song lyrics from the HTML
    return html.find("div", class_="lyrics").get_text()

***GETTING NEGATIVE WORDS***

dictionary of words is score_word_dict

In [50]:
#Get requests response "Failed to establish a new connection: [Errno -2]"
url_web = "https://raw.githubusercontent.com/hyharryhuang/AFINN-json/master/AFINN.json"
#We are going to hardcode the Afinn-111 json

afinn_data = rq.get(url_web).json()

In [51]:
score_word_dict = dict(afinn_data) #negative word dictionary

In [52]:
#add firearms
firearm_url = "https://genius.com/Rap-genius-firearms-and-firearm-companies-mentioned-in-rap-lyrics"
scrape_words = getLyrics(firearm_url)
firearm_list = scrape_words[130:].split("\n")

#add firearms in dictionary with sentiment score of -4
for firearm in firearm_list:
    score_word_dict[firearm.lower()] = -4

In [53]:
#add bulleted lists on rapgenius to a list that is returned and adds to score_word_dict
#needs score_word_dict to word, and adds the sentiment of that word (number)
def add_bulleted_lists(URL, number):
    words_drug = getLyrics(URL)
    first_list = words_drug.split("\n")
    new_list = []

    for x in first_list:
        new_list.append(x.replace(u'\u2022 ', ''))

    #add drugs in dictionary with sentiment score of -1
    for thing in new_list:
        score_word_dict[thing.lower()] = number

In [54]:
slang_drugs_url = "https://genius.com/Rap-genius-slang-dictionary-drugs-lyrics"
slang_firearms_url = "https://genius.com/Rap-genius-slang-dictionary-firearms-lyrics"

#adds words into sentiment dictionary
add_bulleted_lists(slang_drugs_url, -2)
add_bulleted_lists(slang_firearms_url, -4)

***REQUEST RAPGENIUS API***

***GET RAPGENIUS LYRICS***

use: get_song_lyrics(artist, song name)

In [270]:
#input: artist and track title as string
#output: lyrics of song track as string
def song_lyrics(artist, track_title):
    track_title = track_title.replace("&", "and").partition("(")[0].strip()
    def _process(text):
        # formats the url
        puncs = set(string.punctuation) - set("-")
        text = [''.join([c for c in s if c not in puncs]).lower() 
                  for s in unidecode.unidecode(text).split()]
        return text
        
    artist = "-".join(_process(artist)).capitalize()
    track_title = "-".join(_process(track_title))
    
    URL = "https://genius.com/{'-'.join([artist, track_title])}-lyrics"
    
    # request page to get markup
    page = rq.get(URL)
    html = BeautifulSoup(page.text, "html.parser")
    lyrics = html.find("div", class_="lyrics").get_text()

    lyrics = " ".join([line for line in lyrics.strip("\n").split("\n") if not line.startswith('[') and line != ""])
    
    return " ".join(_process(lyrics))

In [145]:
#**supports get_song_lyrics**
#cleans up lyrics once it gets sent back from rapgenius
#pass parameter as string lyrics
#returns as string of lyrics separated by spaces
def process_lyrics (lyrics):
    #removes stuff in brackets
    new_lyrics = re.sub("\[(.+?)\]", '', lyrics)
    
    #removes special characters and newline
    no_char = ''.join(e for e in new_lyrics if e.isalnum() or e == " " or e == "\n")
    no_char = " ".join(no_char.split("\n"))
    
    #returns string in lowercase
    return no_char.lower()

***ANALYZE SENTIMENT SCORE FOR LYRICS***

In [153]:
#input: artist as string and song name as string
#output: integer sentiment score for song
#necessary: needs get_song_lyrics and process_lyrics as auxillary methods
def analyze_senti(artist, song):
    lyrics_list = song_lyrics(artist, song).split(" ")
    
    value = 0
    senti = 0
    for word in lyrics_list:
        if word in score_word_dict:
            value = value + int(score_word_dict[word])
        else:
            continue
        senti = value / (1.0 * len(lyrics_list))
    return senti

In [269]:
#input: song as lyrics as string
#output: sentiment score as int
def sentimentAnalysis(song):
    lyrics_list = song.split(" ")
    value = 0
    senti = 0
    for word in lyrics_list:
        if word in score_word_dict:
            value = value + int(score_word_dict[word])
        else:
            continue
        senti = value / (1.0 * len(lyrics_list))
    return senti

In [154]:
#input: artist as string for artist name, list of songs in album as list
#output: dict of album with senti scores
def process_album(artist, album):
    total = 0
    album_dict = {}
    for song in album:
        album_dict[song] = analyze_senti(artist, song)
    return album_dict

In [155]:
#input: artist as string and discography where album is key and album songs are list as value
#output: dict of artist discography where key is album and value is dictionary with songs as keys and senti scores as values 
def process_artist(artist, discography):
    total = 0
    artist_dict = {}
    for key, value in discography.iteritems():
        artist_dict[key] = process_album(artist, value)
    return artist_dict

In [156]:
#input: artist as string
#output: discography as dictionary where album are keys and songs are list of values
def get_disc(artist):
    url = "data/albums/%s.txt" % artist.decode("utf-8")
    with open(url, 'r') as f:
        x = eval(f.read())
    discography = {}
    for album in x: #x[0]
        discography[album["name"]] = [re.sub("\([^)]*\)", "", track['track_title']) for track in album['tracks']]
    return discography

In [157]:
def get_rappers():
    
    def validate(name):
        # checks if the name has a
        illegal_chars = "|?:^*\""

        for c in illegal_chars:
            if c == '|':
                if name.find(c) > 0:
                    name = name.split("|")[1]
            if c == "\"":
                if name.find(c) > 0:
                    name = name.replace(r'"','',2)
            else: 
                name = name.replace(c,'')
        return name
    
    with open("data/rappers.txt", "r") as f:
        rappers = f.read()
        
    return [validate(rapper).strip() for rapper in rappers.split("\n")]
rappers = get_rappers() 

In [159]:
def save_lyrics(artist, discography):
    total = 0
    artist_dict = {}
    for key, value in discography.iteritems():
        artist_dict[key] = process_album(artist, value)
    return artist_dict

In [91]:
def process_album(artist, album):
    total = 0
    album_dict = {}
    for song in album:
        album_dict[song] = get_song_lyrics(artist, song)
    url = "data/lyrics/%s.txt" % artist
    file = open(url, 'w')
    file.write(str(album_dict))
    file.close()

In [ ]:
#input rapper name
#output overall aggression score
def ov_aggression(artist):
    art_dict = process_artist(artist)
    total = 0
    for value in art_dict.iteritems():
        sum_of_all_album = sum(value.values())
        total = total + (sum_of_all_album / (1.0 * len(value)))
    return total / (1.0 * len(art_dict))

In [168]:
def get_bf1(artist):
    with open("data/lyrics/%s.txt" % artist, "r") as f:
        info = eval(f.read())
    big_string = ""
    for album in info:
        for track in album["tracks"]:
            big_string += track["lyrics"]
    return big_string

In [258]:
get_bf1("A$AP Rocky")

''

In [294]:
dict_senti = {}
for rapper in x:
    print rapper
    try:
        dict_senti[rapper] = sentimentAnalysis(get_bf1(rapper))
    except SyntaxError, IOError:
        continue
    

100 Kila
100s
12 Gauge
2 Chainz
2 Pistols
2$ Fabo
21 Savage
2Mex
360
40 Glocc
50 Cent
6ix9ine
6lack
The 6th Letter
9th Wonder
Ab-Soul
A Boogie wit da Hoodie
Abstract Rude
Ace Hood
Aceyalone
Action Bronson
Adam Saleh
Aesop Rock
Afrika Bambaataa
Afroman
Afu-Ra
Agallah
Ahmad Lewis
Ajs Nigrutin
AKA
Akala
Akinyele
Akir
Akon
The Alchemist
Alex Wiley
Ali Vegas
AMG
Amil
Ampichino
Anderson .Paak
Andre Nickatina
Andy Mineo
Angel Haze
Angie Martinez
Ant Banks
Antoinette
Apache
A$AP Ferg
A$AP Nast
A$AP Rocky
A$AP Yams
A$ton Matthews
Asher Roth
Astronautalis
Awol One
Awkwafina
AZ
Azealia Banks
B-Legit
B-Real
B.G.
B.G. Knocc Out
B.o.B
Baby Bash
Baby Boy da Prince
Baby D
Bad Azz
Badshah
Baeza
Bahamadia
Baka Not Nice
Bangladesh
Bas
Battlecat
Beanie Sigel
Becky G
Benny Blanco
Beenzino
Benzino
BewhY
Bhad Bhabie
Big Boi
Big Daddy Kane
Big Ed
Big Gipp
Big Hawk
Big K.R.I.T.
Big Kuntry King
Big L
Big Lurch
Big Mello
Big Mike
Big Moe
Big Noyd
Big Pokey
Big Pooh
Big Pun
Big Reese
Big Scoob
Big Smo
Big Sean
Bi

IOError: [Errno 2] No such file or directory: 'data/lyrics/Cashis.txt'

In [295]:
dict_senti

removelist = []
for key, value in dict_senti.iteritems():
    if value == -4:
        removelist.append(key)
    
for artist in removelist:
    del dict_senti[artist]

In [297]:
len(dict_senti)

120

In [298]:
f = open("data/senti.txt","w")
f.write(str(dict_senti))
f.close()

In [288]:
!git pull origin master

Updating a599cb7..76e6fdc
Fast-forward
 data/lyrics/Canibus.txt       |   1 +
 data/lyrics/Capital Steez.txt |   1 +
 data/lyrics/Capone.txt        |   1 +
 data/lyrics/Cappadonna.txt    |   1 +
 data/lyrics/Cardi B.txt       |   1 +
 data/lyrics/Casey Veggies.txt |   0
 get_lyrics.ipynb              | 385 +++++++++++++++++++++++++++++++++++++++++-
 7 files changed, 389 insertions(+), 1 deletion(-)
 create mode 100644 data/lyrics/Capital Steez.txt
 create mode 100644 data/lyrics/Capone.txt
 create mode 100644 data/lyrics/Cappadonna.txt
 create mode 100644 data/lyrics/Cardi B.txt
 create mode 100644 data/lyrics/Casey Veggies.txt


From https://github.com/allengueco/rap_aggression
 * branch            master     -> FETCH_HEAD
   a599cb7..76e6fdc  master     -> origin/master


In [284]:
!git add .

The file will have its original line endings in your working directory.


In [285]:
!git commit -m "new methods to analyze big string and senti.txt file as dict"

[master a599cb7] new methods to analyze big string and senti.txt file as dict
 9 files changed, 437 insertions(+), 101 deletions(-)
 create mode 100644 data/lyrics/.ipynb_checkpoints/A$AP Rocky-checkpoint.txt
 create mode 100644 data/lyrics/.ipynb_checkpoints/Boosie Badazz-checkpoint.txt
 create mode 100644 data/lyrics/.ipynb_checkpoints/Bow Wow-checkpoint.txt
 create mode 100644 data/lyrics/.ipynb_checkpoints/Butch Cassidy-checkpoint.txt
 create mode 100644 data/lyrics/.ipynb_checkpoints/C-Bo-checkpoint.txt
 delete mode 100644 data/lyrics/.ipynb_checkpoints/drake-checkpoint.txt
 delete mode 100644 data/lyrics/drake.txt
 create mode 100644 data/senti.txt


In [286]:
!git push

To https://github.com/allengueco/rap_aggression.git
   4b0a272..a599cb7  master -> master
